You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the rapid advancements in artificial intelligence technology
- Mention of key players in the industry and recent noteworthy news in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Emphasis on machine learning, deep learning, natural language processing, and computer vision
- Impact of AI in various industries such as healthcare, finance, and marketing
- Rise of autonomous vehicles and sma

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial intelligence (AI) has been making significant strides in recent years, revolutionizing various industries and shaping the future of technology. With advancements in machine learning, deep learning, natural language processing, and computer vision, AI is becoming more integrated into our daily lives. Key players in the industry such as Google's DeepMind, IBM Watson, and Microsoft Cognitive Services are leading the way in AI innovation. Recent noteworthy news in AI includes breakthroughs in research, ethical considerations, and regulatory developments that are shaping the landscape of artificial intelligence.

## Latest Trends in Artificial Intelligence
Machine learning, deep learning, natural language processing, and computer vision are at the forefront of AI innovation. These technologies are driving advancements in healthcare, finance, marketing, 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial intelligence (AI) has been making significant strides in recent years, revolutionizing various industries and shaping the future of technology. With advancements in machine learning, deep learning, natural language processing, and computer vision, AI is becoming more integrated into our daily lives. Key players in the industry such as Google's DeepMind, IBM Watson, and Microsoft Cognitive Services are leading the way in AI innovation. Recent noteworthy news in AI includes breakthroughs in research, ethical considerations, and regulatory developments that are shaping the landscape of artificial intelligence.

## Latest Trends in Artificial Intelligence
Machine learning, deep learning, natural language processing, and computer vision are at the forefront of AI innovation. These technologies are driving advancements in healthcare, finance, marketing, and other industries, improving efficiency and decision-making processes. The rise of autonomous vehicles and smart home devices powered by AI further demonstrates the impact of artificial intelligence on our daily lives, making tasks more convenient and enhancing user experiences.

## Key Players in the AI Industry
Tech giants like Google, IBM, and Microsoft are leading players in the AI industry, with their cutting-edge technologies and research initiatives driving innovation. Startups such as OpenAI, Cognilytica, and DataRobot are also making waves in AI, bringing fresh perspectives and solutions to the table. Collaboration between these tech giants and research institutions is crucial for advancing AI capabilities and pushing the boundaries of what is possible in artificial intelligence.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI research, such as AlphaGo defeating human champions and the development of the GPT-3 language model, have showcased the immense potential of artificial intelligence. However, ethical considerations surrounding AI development and deployment are becoming increasingly important, prompting discussions on responsible AI use. Regulations and policies are also shaping the future of AI technology, ensuring that it is used ethically and responsibly to benefit society.

## Conclusion
As artificial intelligence continues to evolve and shape the future of technology, staying informed about the latest trends, key players, and noteworthy news in AI is essential. By exploring further resources on AI advancements, engaging in discussions about the ethical implications of AI, and staying updated on the latest trends in artificial intelligence, readers can empower themselves to make informed decisions in this rapidly evolving technological landscape.

Remember to explore reputable resources such as Gartner's AI Trends Report, Forbes articles on top AI companies, and MIT Technology Review for in-depth analysis of AI breakthroughs. By keeping up-to-date with industry-specific publications and reports from organizations like the World Economic Forum, readers can gain valuable insights into the world of artificial intelligence and its impact on society.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "discipline is better than motivation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on discipline is better than motivation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: Why Discipline is Better than Motivation: A Comprehensive Guide

Outline:
I. Introduction
- Definition of discipline and motivation
- Brief overview of the debate on discipline vs. motivation
- Thesis statement: Arguing why discipline is superior to motivation

II. The Latest Trends and Key Players
- Recent studies and articles supporting the importance of discipline
- Quotes from experts in the field of psychology and self-improvement
- Examples of successful individuals attributing their

I now can give a great answer

Final Answer: 

# Why Discipline is Better than Motivation: A Comprehensive Guide

## Introduction
In the realm of personal growth and self-improvement, the debate between discipline and motivation has been ongoing. While both are crucial for success, discipline stands out as the superior force that drives individuals towards their goals. Discipline can be defined as the ability to consistently follow a set of rules or routines, while motivation is the desire or willingness to take action towards achieving a goal. In this blog post, we will delve into why discipline is better than motivation and how it plays a pivotal role in long-term success.

## The Latest Trends and Key Players
Recent studies and articles have shed light on the importance of discipline in achieving success. Experts in the fields of psychology and self-improvement have emphasized the role of discipline in creating lasting change. Successful individuals across various industries have at

In [16]:
Markdown(result)

# Why Discipline is Better than Motivation: A Comprehensive Guide

## Introduction
In the realm of personal growth and self-improvement, the debate between discipline and motivation has been ongoing. While both are crucial for success, discipline stands out as the superior force that drives individuals towards their goals. Discipline can be defined as the ability to consistently follow a set of rules or routines, while motivation is the desire or willingness to take action towards achieving a goal. In this blog post, we will delve into why discipline is better than motivation and how it plays a pivotal role in long-term success.

## The Latest Trends and Key Players
Recent studies and articles have shed light on the importance of discipline in achieving success. Experts in the fields of psychology and self-improvement have emphasized the role of discipline in creating lasting change. Successful individuals across various industries have attributed their achievements to their unwavering discipline and commitment to their goals. These key players in the realm of personal development serve as living examples of the power of discipline in driving results.

## Target Audience Analysis
Our target audience primarily consists of young adults aged 18-35 who are seeking personal growth and self-improvement. These individuals are interested in self-help, productivity, goal-setting, and success stories. They often struggle with motivation, lack consistency in their efforts, and feel overwhelmed or stuck in their pursuit of success. By highlighting the benefits of discipline over motivation, we aim to provide valuable insights and practical tips to help our audience overcome their challenges.

## Key Points

### 1. Discipline is sustainable, while motivation is temporary
Discipline is the foundation of long-term success, as it involves consistently taking action towards one's goals, even when motivation wanes. Research has shown that individuals who prioritize discipline over motivation are more likely to achieve their desired outcomes. Case studies of successful individuals highlight the power of discipline in overcoming obstacles and staying focused on their objectives.

### 2. Discipline builds habits, motivation relies on emotions
Discipline plays a crucial role in forming healthy habits that contribute to personal growth and success. While motivation can serve as a catalyst for action, it often fluctuates based on emotions. Developing discipline allows individuals to maintain a routine and stay committed to their goals, even when faced with challenges. By cultivating discipline, individuals can create sustainable habits that lead to long-lasting results.

### 3. Discipline fosters resilience and perseverance
One of the key benefits of discipline is its ability to build resilience and perseverance in the face of adversity. By staying disciplined and consistent in their efforts, individuals can overcome setbacks and challenges more effectively. Consistency is essential in achieving long-term goals, and discipline plays a critical role in ensuring that individuals stay on track despite the obstacles they may encounter. Strategies for staying disciplined, such as creating a routine and setting achievable goals, can help individuals navigate challenges and stay focused on their objectives.

## Call to Action
We encourage our readers to prioritize discipline over motivation in their daily lives to achieve lasting success. By taking actionable steps to develop discipline and stay consistent in their efforts, individuals can overcome their challenges and reach their full potential. We invite our audience to share their own success stories or challenges with discipline, creating a community of support and inspiration for personal growth and self-improvement.

In conclusion, discipline surpasses motivation as the driving force behind success. By understanding the importance of discipline, individuals can cultivate the mindset and habits needed to achieve their goals. Through consistent effort and unwavering commitment, discipline can lead to long-term success and personal fulfillment.

## SEO Keywords
- Discipline vs. motivation
- Importance of discipline in success
- Building habits through discipline
- Overcoming challenges with discipline

## Resources
- Books: "Atomic Habits" by James Clear, "The Compound Effect" by Darren Hardy
- Websites: Psychology Today, Harvard Business Review
- Studies: Stanford University study on the impact of discipline on goal achievement

By following this comprehensive content plan, we aim to provide valuable insights and practical tips for our target audience on why discipline is better than motivation.

In [17]:
topic = "Transformer Architecture"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Transformer Architecture.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: Demystifying Transformer Architecture: A Comprehensive Guide

Outline:
I. Introduction
- Definition of Transformer Architecture
- Brief history and evolution
- Importance and applications in AI and machine learning

II. Latest Trends in Transformer Architecture
- Attention mechanisms and self-attention
- Pre-training and fine-tuning models
- Integration with other deep learning techniques

III. Key Players in Transformer Architecture
- Google's BERT (Bidirectional Encoder Representation

I now can give a great answer

Final Answer:
# Demystifying Transformer Architecture: A Comprehensive Guide

## Introduction

Transformer Architecture is a groundbreaking approach in the realm of artificial intelligence and machine learning. By utilizing attention mechanisms and self-attention, it effectively processes sequential data in a more efficient manner compared to traditional models. The evolution of the concept of Transformers has led to significant advancements in natural language processing, computer vision, and various other AI applications.

## Latest Trends in Transformer Architecture

One of the prominent trends in Transformer Architecture is the focus on attention mechanisms and self-attention, which allow models to concentrate on relevant parts of the input sequence, thereby enhancing performance and accuracy. Moreover, techniques such as pre-training and fine-tuning have gained popularity in optimizing Transformer models for specific tasks. Integration with other dee

In [18]:
Markdown(result)

# Demystifying Transformer Architecture: A Comprehensive Guide

## Introduction

Transformer Architecture is a groundbreaking approach in the realm of artificial intelligence and machine learning. By utilizing attention mechanisms and self-attention, it effectively processes sequential data in a more efficient manner compared to traditional models. The evolution of the concept of Transformers has led to significant advancements in natural language processing, computer vision, and various other AI applications.

## Latest Trends in Transformer Architecture

One of the prominent trends in Transformer Architecture is the focus on attention mechanisms and self-attention, which allow models to concentrate on relevant parts of the input sequence, thereby enhancing performance and accuracy. Moreover, techniques such as pre-training and fine-tuning have gained popularity in optimizing Transformer models for specific tasks. Integration with other deep learning techniques further amplifies the capabilities of Transformer Architecture across different domains.

## Key Players in Transformer Architecture

Google's BERT, OpenAI's GPT, and Facebook's RoBERTa stand out as key players driving innovation in Transformer Architecture. These models have showcased state-of-the-art performance in natural language processing tasks, underscoring the efficacy of Transformer-based approaches. Each model offers distinct features and capabilities, making them invaluable assets within the AI and machine learning community.

## Noteworthy News in Transformer Architecture

Recent advancements in model efficiency have propelled Transformer Architecture to new heights, with applications in natural language processing and computer vision expanding rapidly. While opportunities for researchers and industry professionals are on the rise, challenges such as model interpretability and scalability persist, emphasizing the importance of continued research and development in the field.

## Target Audience Analysis

The primary audience for Transformer Architecture content includes data scientists, machine learning engineers, researchers, academics in AI, and tech enthusiasts. By catering to this diverse group, valuable insights and resources can be provided to individuals interested in deep learning technologies. Keeping abreast of the latest trends and developments in Transformer Architecture is crucial for driving innovation and progress in the field.

## SEO Keywords

Transformer Architecture, Attention Mechanisms, BERT, GPT, RoBERTa, Machine Learning Trends, Deep Learning Techniques

## Resources

For those looking to delve deeper into Transformer Architecture, exploring research papers, official documentation from key players, online courses, and industry reports is recommended. Staying informed about the latest developments and contributing to the open-source community can help enhance knowledge and skills in AI and machine learning.

In conclusion, Transformer Architecture represents a cutting-edge technology with immense potential in AI and machine learning. By grasping the latest trends, key players, and notable news in the field, the power of Transformers can be harnessed to drive innovation and create impactful solutions for real-world challenges. Stay informed, stay curious, and stay ahead in the world of Transformer Architecture.

In [19]:
topic = "5 Bed Time stories of kids below 1 year"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on 5 Bed Time stories of kids below 1 year.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: 5 Bed Time Stories for Kids Below 1 Year

Introduction:
- Brief overview of the importance of bedtime stories for infants' cognitive development and emotional bonding with parents.
- Mention the benefits of incorporating storytelling into the bedtime routine for babies.

Key Points:
1. "Goodnight Moon" by Margaret Wise Brown
- Summary of the classic bedtime story and its soothing effect on babies.
- Highlight the repetitive text and simple illustrations that engag

I now can give a great answer

Final Answer:
# 5 Bed Time Stories for Kids Below 1 Year

## Introduction
Bedtime stories play a vital role in the cognitive development and emotional bonding of infants with their parents. Incorporating storytelling into the bedtime routine not only helps babies relax and prepare for sleep but also fosters a strong connection between parents and their little ones. The soothing effect of bedtime stories can create a calming atmosphere perfect for bedtime, making it an essential part of a child's nightly routine.

## 1. "Goodnight Moon" by Margaret Wise Brown
"Goodnight Moon" is a timeless bedtime story known for its calming effect on babies. The repetitive text and simple illustrations engage infants, creating a serene environment ideal for bedtime. The soothing rhythm of the story helps babies relax and transition to sleep, making it a must-have in every parent's bedtime story collection.

## 2. "Time for Bed" by Mem Fox
"Time for Bed" by Mem Fox feature

In [21]:
Markdown(result)

# 5 Bed Time Stories for Kids Below 1 Year

## Introduction
Bedtime stories play a vital role in the cognitive development and emotional bonding of infants with their parents. Incorporating storytelling into the bedtime routine not only helps babies relax and prepare for sleep but also fosters a strong connection between parents and their little ones. The soothing effect of bedtime stories can create a calming atmosphere perfect for bedtime, making it an essential part of a child's nightly routine.

## 1. "Goodnight Moon" by Margaret Wise Brown
"Goodnight Moon" is a timeless bedtime story known for its calming effect on babies. The repetitive text and simple illustrations engage infants, creating a serene environment ideal for bedtime. The soothing rhythm of the story helps babies relax and transition to sleep, making it a must-have in every parent's bedtime story collection.

## 2. "Time for Bed" by Mem Fox
"Time for Bed" by Mem Fox features gentle language and rhythmic patterns that aid in calming babies and preparing them for sleep. The comforting story helps infants wind down and get ready for bedtime, making it a perfect choice for the nightly ritual. The emotional connection that this story fosters between parent and child is invaluable for creating a strong bond.

## 3. "Guess How Much I Love You" by Sam McBratney
"Guess How Much I Love You" is a heartwarming tale of love and affection between a parent and child. Reading this story to babies not only strengthens the emotional bond but also instills a sense of security and love. The emotional resonance of this story makes it a cherished part of bedtime routines for many families.

## 4. "Baby Beluga" by Raffi
"Baby Beluga" is a musical book featuring a beloved children's song. The interactive elements in the story engage babies during storytime, providing a fun and entertaining experience for both parents and infants. The joy and excitement that this story brings can create lasting memories for families and make bedtime a delightful time of bonding.

## 5. "The Going-To-Bed Book" by Sandra Boynton
"The Going-To-Bed Book" offers a humorous and playful take on the bedtime routine, captivating infants with its interactive features. The engaging story enhances the reading experience, making it a delightful addition to the bedtime story collection. The fun and light-hearted nature of this book can make bedtime a joyful and entertaining experience for both parents and babies.

## Call to Action
I urge parents to incorporate these 5 bedtime stories into their daily routine with their babies. Creating a cozy and calming environment for bedtime storytelling can enhance the bonding experience and create lasting memories for both parents and infants. By following these recommendations, parents can establish a meaningful bedtime routine that promotes cognitive development and strengthens the emotional connection with their little ones.

Remember, bedtime stories are not just for entertainment; they are a valuable tool for nurturing the parent-child relationship and fostering a love for reading from an early age. So, snuggle up with your baby, pick up a book, and immerse yourselves in the magical world of storytelling together!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).